In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 설치
!pip install transformers
!pip install sentencepiece
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install transformers torch

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-2o4tr_uj/kobert-tokenizer_c8991f3736314d3ab21b0c888315a046
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-2o4tr_uj/kobert-tokenizer_c8991f3736314d3ab21b0c888315a046
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=b3c8639acf1ab4b20bcebc0230ab52aa33fc1645930c13232586ef5aea8450e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-zvamw9hp/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
# 모듈 불러오기
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from kobert_tokenizer import KoBERTTokenizer  # 대문자 'BERT'로 수정
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from itertools import product

In [ ]:
# 데이터 불러오기
reviews_df = pd.read_csv('/content/drive/MyDrive/Final/전처리/불용어_처리/spelled_bareun_final.csv')

## max_len

In [ ]:
# 문장 길이
lengths = get_sentence_length_distribution(reviews_df['new_content_cleaned_spelled'])

# 가장 긴 값
max_len = max(lengths)
max_len

156

# Hugging

In [ ]:
# 감성 라벨이 있는 데이터만 필터링 (NaN 값 제거)
#auto_train_labeled = auto_train[auto_train['sentiment_label'].notna()]
manual_train_labeled = manual_train[manual_train['sentiment_label'].notna()]
test_labeled = test[test['sentiment_label'].notna()]

In [ ]:
# 라벨 값 변환: -1 -> 0 (부정), 0 -> 1 (중립), 1 -> 2 (긍정)
manual_train_labeled['sentiment_label'] = manual_train_labeled['sentiment_label'].map({-1: 0, 0: 1, 1: 2})
test_labeled['sentiment_label'] = test_labeled['sentiment_label'].map({-1: 0, 0: 1, 1: 2})

# 만약 auto_train_labeled에도 라벨이 존재한다면 동일하게 적용
auto_train['sentiment_label'] = auto_train['sentiment_label'].map({-1: 0, 0: 1, 1: 2})

<ipython-input-10-a10e0db41651>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auto_train['sentiment_label'] = auto_train['sentiment_label'].map({-1: 0, 0: 1, 1: 2})


In [ ]:
# Hugging Face의 KoBERT 모델과 KoBERT 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=3)

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(next(model.parameters()).device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This beha

cuda:0


In [ ]:
# 데이터셋 정의
class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, has_labels=True):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.has_labels = has_labels  # 레이블이 있는지 여부 확인

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        review = self.df.iloc[index]['new_content_cleaned_spelled']

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

        # 레이블이 있는 경우에만 labels 반환
        if self.has_labels:
            label = self.df.iloc[index]['sentiment_label']
            item['labels'] = torch.tensor(label, dtype=torch.long)

        return item

In [ ]:
# score 계산 함수
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# KoBERT 토크나이저 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

# 튜닝할 하이퍼파라미터 설정
learning_rates = [1e-5, 2e-5]
batch_sizes = [8]
num_train_epochs = [2, 4]
weight_decays = [0.01, 0.1]

# Grid Search를 위한 하이퍼파라미터 조합 생성
param_grid = list(product(learning_rates, batch_sizes, num_train_epochs, weight_decays))

# 결과 저장을 위한 리스트
results_list = []

# 최적의 F1 스코어와 모델을 저장하기 위한 변수 초기화
best_f1_score = 0
best_model = None
best_model_path = '/content/drive/MyDrive/best_model'

# Grid Search 방식으로 각 하이퍼파라미터 조합에 대해 학습 및 평가
for params in param_grid:
    learning_rate, batch_size, num_epochs, weight_decay = params
    print(f"Training with lr={learning_rate}, batch_size={batch_size}, epochs={num_epochs}, weight_decay={weight_decay}")

    # 모델 초기화: 하나의 하이퍼파라미터 조합마다 새로운 모델 생성
    model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=3)

    # StratifiedKFold 설정: app_name을 기준으로 5개의 폴드로 나눔, random_state=42로 설정
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # 각 fold의 성능 결과 저장을 위한 리스트
    accuracy_list = []
    f1_list = []

    # fold 별로 데이터 나누고 학습 및 검증
    for fold, (train_index, val_index) in enumerate(skf.split(manual_train_labeled, manual_train_labeled['sentiment_label'])):
        print(f"Fold {fold + 1}")

        # train_index와 val_index를 사용해 데이터를 나눕니다.
        train_data = manual_train_labeled.iloc[train_index]
        val_data = manual_train_labeled.iloc[val_index]

        # 각 fold마다 새로운 Dataset을 만듭니다.
        train_dataset = SentimentDataset(train_data, tokenizer, max_len=max_len)
        val_dataset = SentimentDataset(val_data, tokenizer, max_len=max_len)

        # TrainingArguments 설정
        training_args = TrainingArguments(
            output_dir=f'./results_lr_{learning_rate}_bs_{batch_size}_epochs_{num_epochs}_wd_{weight_decay}_fold_{fold + 1}',
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            warmup_steps=500,
            weight_decay=weight_decay,
            logging_dir=f'./logs_lr_{learning_rate}_bs_{batch_size}_epochs_{num_epochs}_wd_{weight_decay}_fold_{fold + 1}',
            logging_steps=10,
            learning_rate=learning_rate,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1"
        )

        # Trainer 설정
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )

        # 모델 학습
        trainer.train()

        # 평가 (각 폴드에서의 성능 저장)
        results = trainer.evaluate()
        accuracy_list.append(results['eval_accuracy'])
        f1_list.append(results['eval_f1'])

    # 각 폴드의 평균 성능 계산
    mean_accuracy = sum(accuracy_list) / len(accuracy_list)
    mean_f1 = sum(f1_list) / len(f1_list)

    # 결과 저장
    result_dict = {
        'learning_rate': learning_rate,
        'batch_size': batch_size,
        'num_epochs': num_epochs,
        'weight_decay': weight_decay,
        'mean_accuracy': mean_accuracy,
        'mean_f1': mean_f1
    }

    results_list.append(result_dict)

    # 만약 현재 모델의 F1 스코어가 최고라면, 그 모델을 저장
    if mean_f1 > best_f1_score:
        best_f1_score = mean_f1
        best_model = model

        # 현재의 최적 모델 저장
        best_model.save_pretrained(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print(f"New best model saved with F1 score: {best_f1_score}")

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results_list)

# 최적의 하이퍼파라미터를 보고 싶은 경우
best_row = results_df.loc[results_df['mean_f1'].idxmax()]
print("Best hyperparameters based on mean F1:")
print(best_row)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Training with lr=1e-05, batch_size=8, epochs=2, weight_decay=0.01


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Fold 1


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.092000,0.188993,0.949170,0.944671,0.940217,0.949170
2,0.089000,0.194248,0.953838,0.949484,0.945184,0.953838


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.032000,0.012944,0.998443,0.998449,0.998473,0.998443
2,0.104900,0.002592,0.999481,0.999488,0.999510,0.999481
3,0.000200,0.009971,0.997924,0.997927,0.997955,0.997924
4,0.000200,0.005368,0.998962,0.998969,0.998991,0.998962


Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.002300,0.009586,0.997924,0.997918,0.997928,0.997924
2,0.000600,0.009406,0.997405,0.997401,0.997417,0.997405
3,0.000100,0.011867,0.997405,0.997401,0.997417,0.997405
4,0.000000,0.012185,0.997924,0.997924,0.997924,0.997924


New best model saved with F1 score: 0.9859557854482773
Best hyperparameters based on mean F1:
learning_rate    0.000020
batch_size       8.000000
num_epochs       4.000000
weight_decay     0.100000
mean_accuracy    0.986203
mean_f1          0.985956
Name: 7, dtype: float64


In [ ]:
results_df = pd.DataFrame(results_list)
results_df

,learning_rate,batch_size,num_epochs,weight_decay,mean_accuracy,mean_f1
0,0.00001,8,2,0.01,0.980186,0.978295
1,0.00001,8,2,0.10,0.975932,0.974177
2,0.00001,8,4,0.01,0.986307,0.985510
3,0.00001,8,4,0.10,0.985477,0.984715
4,0.00002,8,2,0.01,0.981638,0.980289
5,0.00002,8,2,0.10,0.978214,0.976393
6,0.00002,8,4,0.01,0.985373,0.984658
7,0.00002,8,4,0.10,0.986203,0.985956


## Auto Train만 이용하여 Fine Tuning

In [ ]:
# KoBERT의 토크나이저 불러오기
best_model_path = '/content/drive/MyDrive/best_model'
tokenizer = KoBERTTokenizer.from_pretrained(best_model_path)

# 저장된 best_model 불러오기
new_model = BertForSequenceClassification.from_pretrained(best_model_path)

In [ ]:
# auto_train 데이터셋을 SentimentDataset으로 변환
auto_train_dataset = SentimentDataset(auto_train, tokenizer, max_len=max_len, has_labels=False)

# 사전 학습된 모델을 사용하여 라벨 없는 데이터에 대한 예측 수행

# 최적의 하이퍼 파라미터 설정
training_args = TrainingArguments(
    output_dir='./results_final',
    num_train_epochs=4,      # 최적의 에폭 수
    per_device_train_batch_size=8,  # 최적의 배치 크기
    learning_rate=2e-5,  # 최적의 학습률
    weight_decay=0.1,    # 최적의 weight decay
    logging_dir='./logs_final',
    evaluation_strategy="no",  # 평가를 생략
    save_strategy="epoch",  # 각 epoch마다 모델을 저장
    load_best_model_at_end=False  # load_best_model_at_end을 끄기
)

# auto_train으로 학습
trainer = Trainer(
    model=new_model,  # 사전 학습된 best model 사용
    args=training_args,
    train_dataset=auto_train_dataset
)

# 예측 수행
predictions = trainer.predict(auto_train_dataset)  # logits 예측

# Step 5: 각 예측에서 가장 높은 확률을 가지는 라벨을 pseudo-label로 지정
pseudo_labels = np.argmax(predictions.predictions, axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# 신뢰도 계산 (각 샘플의 최대 확률값을 신뢰도로 간주)
confidences = np.max(predictions.predictions, axis=1)

# 신뢰도 임계값 설정
confidence_threshold = 0.8
high_confidence_indices = np.where(confidences >= confidence_threshold)[0]

# 신뢰도가 높은 샘플만 선택하여 새로운 auto_train 데이터셋 생성
high_confidence_auto_train = auto_train.iloc[high_confidence_indices].copy()

# pseudo-label을 신뢰도가 높은 샘플에 추가
high_confidence_auto_train['sentiment_label'] = pseudo_labels[high_confidence_indices]

# Pseudo-label이 포함된 새로운 Dataset 생성
auto_train_dataset_with_confident_labels = SentimentDataset(
    high_confidence_auto_train, tokenizer, max_len=max_len, has_labels=True
)

In [ ]:
# 신뢰도 높은 pseudo-label 데이터로 모델 Fine-tuning
trainer = Trainer(
    model=new_model,
    args=training_args,
    train_dataset=auto_train_dataset_with_confident_labels  # 신뢰도 높은 데이터셋으로 학습
)

# 모델 학습 수행
trainer.train()

Step,Training Loss
500,0.097900
1000,0.103000
1500,0.108700
2000,0.088300
2500,0.087600
3000,0.116700
3500,0.082500
4000,0.079200
4500,0.088500
5000,0.065200


TrainOutput(global_step=19276, training_loss=0.04196019135809806, metrics={'train_runtime': 2427.5978, 'train_samples_per_second': 63.518, 'train_steps_per_second': 7.94, 'total_flos': 1.2361487701870944e+16, 'train_loss': 0.04196019135809806, 'epoch': 4.0})

In [ ]:
# test set에 mapping
test['sentiment_label'] = test['sentiment_label'].map({-1: 0, 0: 1, 1: 2})

<ipython-input-14-5531978ef045>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['sentiment_label'] = test['sentiment_label'].map({-1: 0, 0: 1, 1: 2})


## test set 예측하기

In [ ]:
# KoBERT의 토크나이저 불러오기
best_model_path = '/content/drive/MyDrive/best_model_second'
tokenizer = KoBERTTokenizer.from_pretrained(best_model_path)

# 저장된 best_model 불러오기
new_model = BertForSequenceClassification.from_pretrained(best_model_path)

# 최적의 하이퍼 파라미터 설정
training_args = TrainingArguments(
    output_dir='./results_final',
    num_train_epochs=4,      # 최적의 에폭 수
    per_device_train_batch_size=8,  # 최적의 배치 크기
    learning_rate=2e-5,  # 최적의 학습률
    weight_decay=0.1,    # 최적의 weight decay
    logging_dir='./logs_final',
    evaluation_strategy="no",  # 평가를 생략
    save_strategy="epoch",  # 각 epoch마다 모델을 저장
    load_best_model_at_end=False  # load_best_model_at_end을 끄기
)

# auto_train으로 학습
trainer = Trainer(
    model=new_model,  # 사전 학습된 best model 사용
    args=training_args,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Test 데이터에 대한 예측 수행
test_dataset = SentimentDataset(test, tokenizer, max_len=max_len, has_labels=True)

# 모델을 사용하여 테스트 데이터에 대한 예측
predictions, labels, _ = trainer.predict(test_dataset)
preds = np.argmax(predictions, axis=1)

# classification_report 출력
print("Classification Report:")
print(classification_report(labels, preds, target_names=['Negative', 'Neutral', 'Positive']))

# confusion_matrix 출력
print("Confusion Matrix:")
print(confusion_matrix(labels, preds))

Classification Report:
              precision    recall  f1-score   support

    Negative       0.81      0.87      0.84       554
     Neutral       0.32      0.15      0.20        60
    Positive       0.98      0.98      0.98      4356

    accuracy                           0.95      4970
   macro avg       0.70      0.67      0.67      4970
weighted avg       0.95      0.95      0.95      4970

Confusion Matrix:
[[ 482   10   62]
 [  21    9   30]
 [  92    9 4255]]


In [ ]:
# 모델 학습 후 저장하는 경로 설정
output_dir = "/content/drive/MyDrive/best_model_second"

# 모델 저장 코드
new_model.save_pretrained(output_dir)

# 토크나이저 저장 코드
tokenizer.save_pretrained(output_dir)

print(f"모델과 토크나이저가 {output_dir}에 저장되었습니다.")

모델과 토크나이저가 /content/drive/MyDrive/best_model_second에 저장되었습니다.


In [ ]:
# auto_train 데이터셋을 SentimentDataset으로 변환
auto_train_dataset = SentimentDataset(auto_train, tokenizer, max_len=max_len, has_labels=False)

# 예측 수행
predictions = trainer.predict(auto_train_dataset)  # logits 예측

# 각 예측에서 가장 높은 확률을 가지는 라벨을 pseudo-label로 지정
pseudo_labels = np.argmax(predictions.predictions, axis=1)

In [ ]:
# 파일 저장
auto_train['sentiment_label'] = pseudo_labels
auto_train['sentiment_label'] = auto_train['sentiment_label'] -1

auto_train.to_csv('/content/drive/MyDrive/auto_train_labeled.csv', encoding='utf-8-sig')